In [8]:
# cd ..

/home/emilian/DataspellProjects/audio-diffusion-pytorch-trainer


In [9]:
from audio_diffusion_pytorch import AudioDiffusionModel
import torch
from ipywidgets import widgets
from audio_diffusion_pytorch import KarrasSchedule, AEulerSampler
from main import utils
import numpy as np
import torchaudio
from IPython.display import Audio, HTML, display
import math

In [10]:
import hydra
hydra.initialize(config_path="..", job_name="test_app")

/tmp/ipykernel_341372/2248803464.py:2: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  hydra.initialize(config_path="..", job_name="test_app")


hydra.initialize()

## Initialize model

In [11]:
device = 'cuda'
cfg = hydra.compose(config_name="config", overrides=["exp=base_slakh_supervised_unconditional_2stem"])
model = hydra.utils.instantiate(cfg.model)
ckpt = torch.load('logs/ckpts/drums_piano_slakh/epoch=606-valid_loss=0.017.ckpt', map_location=device)
model.load_state_dict(ckpt['state_dict'])
model : AudioDiffusionModel = model.model.to(device)

## Forward Inference

#### Total generation

In [24]:
sampling_rate = 22050
length = 6
length_samples = 2**math.ceil(math.log2(length * sampling_rate))
num_samples = 2
num_steps = 100
num_stems = len(cfg.stems)

noise = torch.randn(num_samples, num_stems, length_samples).to(device)
samples = model.sample(noise=noise, num_steps=num_steps)

res = {}
for i, stem in enumerate(cfg.stems):
    res[stem] = samples[:, i, :].detach().cpu()
res['mix'] = samples.sum(dim=1).detach().cpu()

for i in range(num_samples):
    audio_widgets = []
    for stem in cfg.stems:
        out = widgets.Output()
        with out:
            display(f'Sample {i} - Stem {stem}')
            display(Audio(res[stem][i, :], rate = sampling_rate))
        audio_widgets.append(out)
    out = widgets.Output()
    with out:
            display(f'Sample {i} - Mixture')
            display(Audio(res['mix'][i, :], rate = sampling_rate))
    audio_widgets.append(out)
    display(widgets.HBox(audio_widgets))

#### Partial generation